In [41]:
import os
import pandas as pd

datasets = ["SMD", "SMAP", "MSL", "UCR", "SWaT", "WADI"]
root = 'processed/'
import numpy as np
res = {}
for dataset in datasets:
    res[dataset] = {}
    all_files = os.listdir(root + dataset)
    entities = []
    for file in all_files:
        if file[-9:] == 'train.npy':
            entities.append(file[:-10])
    res[dataset]['entity'] = len(entities)
    total_size = 0
    dims = set()
    for file in all_files:
        if file[-9:] == 'train.npy':
            size = np.load(root+dataset+'/'+file).shape[0]
            dim = np.load(root+dataset+'/'+file).shape[1]
            total_size += size
            dims.add(dim)

    res[dataset]['train_size'] = total_size
    res[dataset]['dim'] = dims.pop()
    
    total_size = 0
    for file in all_files:
        if file[-8:] == 'test.npy':
            size = np.load(root+dataset+'/'+file).shape[0]
            total_size += size
    res[dataset]['test_size'] = total_size

    total_size = 0
    for file in all_files:
        if file[-10:] == 'labels.npy':
            labels = np.load(root+dataset+'/'+file)
            labelsFinal = (np.sum(labels, axis=1) >= 1) + 0
            total_size += labelsFinal.sum()
    res[dataset]['label_rate'] = round(total_size / res[dataset]['test_size'] * 100, 2)

df = pd.DataFrame(res).T
df.index.name = 'dataset'
df[['entity', 'dim', 'train_size', 'test_size', 'label_rate']].sort_index().to_excel("other_results/datasets_info.xlsx")